In [1]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [8]:
# Importa "HTML e Markdown"

from IPython.display import HTML, Markdown

In [10]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Qual é a empresa responsavel por desenvolver o Gemini?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 O Gemini é uma família de modelos de linguagem grandes multimodais desenvolvidos pelo Google DeepMind.

In [11]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['quem desenvolveu o Gemini?', 'Empresa desenvolvedora do Gemini']
Páginas utilizadas na resposta: wikipedia.org



In [12]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [13]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [14]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [15]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [16]:
##########################################
# --- Agente 1: Buscador de Jogos --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca lista de jogos eletrônicos",
        tools=[google_search],
        instruction="""
          Você é um agente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search) para buscar uma lista com os melhores jogos do ano anterior.
          Foque em no máximo 5 melhores jogos, separados por categorias.
          Exclua jogos que possuem temas eróticos.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [17]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
          Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscador, você deve:
          usar a ferramenta de busca do Google (google_search) para criar um plano sobre quais são os jogos mais populares do ano anterior, você pode usar o (google_search) para encontrar a lista mais relevante com base nas suas pesquisas e retornar esse tema.
        """,
        description="Agente que planeja posts sobre a lista de jogos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [18]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você vai escrever um resumo sobre os jogos mais populares do ano anterior.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para um website sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final para compartilhamentos em redes sociais.
            """,
        description="Agente redator de posts engajadores para website"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [19]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts sobre jogos eletrônicos, com foco em websites e blogs.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de website e blog abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para websites e blogs."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [22]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Websites e Blogs com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite qual CATEGORIA DE JOGOS quer criar o post. Será gerado um TOP 5 para o website ou blog: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar a categoria!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)

  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Websites e Blogs com 4 Agentes 🚀
❓ Por favor, digite qual CATEGORIA DE JOGOS quer criar o post. Será gerado um TOP 5 para o website ou blog: terror
Maravilha! Vamos então criar o post sobre novidades em terror

--- Resultado do Agente 1 (Buscador) ---



> Para criar uma lista dos melhores jogos de terror do ano passado (2024), vou realizar algumas pesquisas no Google para ver quais títulos foram mais bem avaliados pela crítica e pelos jogadores.
> 
> 
> Com base nas minhas buscas, aqui estão alguns dos melhores jogos de terror de 2024, divididos por categorias:
> 
> **Melhor Remake:**
> 
> *   **Silent Hill 2:** Um remake do clássico do PlayStation 2, este jogo promete uma experiência aterrorizante com visuais e áudio aprimorados.
> 
> **Melhores Jogos Inovadores:**
> 
> *   **Crow Country:** Um jogo de terror indie com visuais retrô e uma atmosfera misteriosa.
> *   **Don't Scream:** Um jogo de terror único que exige que você fique em silêncio enquanto explora uma floresta assustadora.
> 
> **Melhor para Jogar com Amigos:**
> 
> *   **The Outlast Trials:** Um jogo de terror cooperativo onde você e seus amigos devem sobreviver a experimentos aterrorizantes.
> 
> **Melhor jogo de terror psicológico:**
> 
> *   **Alan Wake 2:** Uma sequência do popular jogo de terror psicológico, Alan Wake 2 promete uma narrativa intrincada e sustos atmosféricos.
> 
> **Outras recomendações:**
> 
> *   **Still Wakes the Deep:**
> 
> Espero que esta lista ajude você a encontrar alguns jogos de terror assustadores para jogar!


--------------------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Olá! Com base nas informações que você coletou sobre os melhores jogos de terror de 2024, podemos criar um plano de conteúdo para redes sociais. Aqui estão algumas ideias de posts que você pode usar:
> 
> **1. Post de lista:**
> 
> *   **Título:** Os Melhores Jogos de Terror de 2024 para Jogar Sozinho ou com Amigos!
> *   **Conteúdo:**
>     *   Breve introdução sobre a popularidade do gênero de terror nos jogos.
>     *   Lista dos jogos mencionados, divididos pelas categorias (Melhor Remake, Melhores Jogos Inovadores, Melhor para Jogar com Amigos, Melhor Jogo de Terror Psicológico, Outras Recomendações).
>     *   Para cada jogo, inclua:
>         *   Nome do jogo
>         *   Uma frase curta sobre o que torna o jogo especial
>         *   Uma imagem ou vídeo (se disponível)
>     *   Encerramento com uma pergunta para engajar o público (ex: "Qual desses jogos você está mais ansioso para jogar?")
> *   **Hashtags:** #jogosdeterror #horror #terror #jogos #games #2024 #recomendações
> 
> **2. Posts individuais sobre cada jogo:**
> 
> *   **Título:** [Nome do Jogo] – O Jogo de Terror que Você Precisa Conhecer!
> *   **Conteúdo:**
>     *   Uma descrição mais detalhada do jogo (história, jogabilidade, atmosfera).
>     *   Destaque os pontos fortes do jogo e por que ele se destaca no gênero.
>     *   Inclua trailers, gameplays ou screenshots.
>     *   Opinião pessoal sobre o jogo (se você jogou).
>     *   Incentive os seguidores a compartilhar suas experiências com o jogo (se já jogaram) ou suas expectativas.
> *   **Hashtags:** #jogosdeterror #[NomeDoJogo] #horror #terror #games #jogos
> 
> **3. Post comparativo:**
> 
> *   **Título:** Clássico vs. Moderno: Qual Remake de Terror é Melhor?
> *   **Conteúdo:**
>     *   Compare o remake de "Silent Hill 2" com o jogo original.
>     *   Destaque as mudanças e melhorias no remake.
>     *   Pergunte aos seguidores qual versão eles preferem e por quê.
> *   **Hashtags:** #SilentHill2 #remake #jogosdeterror #horror #terror #jogos #games #comparativo
> 
> **4. Post de enquete:**
> 
> *   **Título:** Qual é o Jogo de Terror Mais Assustador de 2024?
> *   **Conteúdo:**
>     *   Crie uma enquete com os jogos da lista.
>     *   Incentive os seguidores a votar e compartilhar suas opiniões nos comentários.
> *   **Hashtags:** #jogosdeterror #enquete #horror #terror #jogos #games #2024
> 
> **5. Post de "você sabia?":**
> 
> *   **Título:** Você Sabia Que... [Fato Interessante Sobre um Jogo de Terror]?
> *   **Conteúdo:**
>     *   Compartilhe um fato curioso sobre um dos jogos da lista (ex: inspirações, desenvolvimento, easter eggs).
>     *   Incentive os seguidores a compartilhar outros fatos que eles conhecem.
> *   **Hashtags:** #jogosdeterror #curiosidades #horror #terror #jogos #games
> 
> **6. Post de vídeo/review:**
> 
> *   **Título:** Minha Opinião Sobre [Nome do Jogo] – Vale a Pena?
> *   **Conteúdo:**
>     *   Crie um vídeo curto com sua opinião sobre o jogo.
>     *   Mostre trechos do jogo e discuta os pontos positivos e negativos.
>     *   Incentive os seguidores a deixar seus comentários e opiniões.
> *   **Hashtags:** #jogosdeterror #review #analise #horror #terror #jogos #games #[NomeDoJogo]
> 
> Para complementar, aqui estão algumas perguntas que usei para pesquisar os jogos mais populares de 2024:
> 
> 
> Perfeito! Aqui estão algumas dicas para otimizar ainda mais seus posts:
> 
> *   **Adapte o conteúdo para cada plataforma:** O que funciona no TikTok pode não funcionar no Instagram. Adapte o formato e o tom para cada rede social.
> *   **Use vídeos curtos e dinâmicos:** Vídeos são mais engajadores do que imagens estáticas. Use trechos de gameplay, trailers ou até mesmo suas próprias reações enquanto joga.
> *   **Interaja com a comunidade:** Responda aos comentários, faça perguntas e incentive os seguidores a compartilhar suas experiências.
> *   **Seja consistente:** Poste regularmente para manter seu público engajado.
> 
> Com este plano e as dicas extras, você estará pronto para criar conteúdo incrível sobre os melhores jogos de terror de 2024!
> 
> Você gostaria de mais ideias ou exemplos específicos de posts?
> 


--------------------------------------------------------------


...

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-67' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

--------------------------------------------------------------


Exception in thread Thread-30 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


--- Resultado do Agente 4 (Revisor) ---



--------------------------------------------------------------
